# pandas的拼接操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

## 0. 回顾numpy的级联

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

In [2]:
n1 = np.random.randint(0,10,size=(3,3))
n2 = np.random.randint(100,200,size=(3,3))
display(n1,n2)

array([[8, 4, 4],
       [1, 5, 9],
       [6, 3, 3]])

array([[114, 136, 125],
       [123, 163, 158],
       [199, 135, 165]])

In [3]:
np.concatenate((n1,n2))

array([[  8,   4,   4],
       [  1,   5,   9],
       [  6,   3,   3],
       [114, 136, 125],
       [123, 163, 158],
       [199, 135, 165]])

In [4]:
np.concatenate((n1,n2),axis=1)

array([[  8,   4,   4, 114, 136, 125],
       [  1,   5,   9, 123, 163, 158],
       [  6,   3,   3, 199, 135, 165]])

为方便讲解，我们首先定义一个生成DataFrame的函数：

## 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
join='outer'
join_axes=None
ignore_index=False
keys = [value1,value2...]
```

In [5]:
def make_df(index,columns):
    dic = {key:[key + str(i) for i in index] for key in columns}
    df = DataFrame(data=dic,index=index)
    return df

In [6]:
df1 = make_df([1,2,3],list('ABC'))
df2 = make_df([1,2,3],list('BCD'))
display(df1,df2)

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3


,B,C,D
1,B1,C1,D1
2,B2,C2,D2
3,B3,C3,D3


### 1)  简单级联

In [7]:
# 默认行级联
pd.concat((df1,df2),sort=True)

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,A3,B3,C3,NaN
1,NaN,B1,C1,D1
2,NaN,B2,C2,D2
3,NaN,B3,C3,D3


和np.concatenate一样，优先增加行数（默认axis=0）

可以通过设置axis来改变级联方向

In [8]:
# 修改级联方向
pd.concat((df1,df2),axis=1,sort=True)

,A,B,C,B,C,D
1,A1,B1,C1,B1,C1,D1
2,A2,B2,C2,B2,C2,D2
3,A3,B3,C3,B3,C3,D3


级连会把该方向上索引相同的元素放在一行（一列），index/columns在级联时可以重复

In [9]:
df3 = make_df([0,1,2],list('AB'))
df4 = make_df([1,2,3],list('AB'))
df3

,A,B
0,A0,B0
1,A1,B1
2,A2,B2


In [10]:
df4

,A,B
1,A1,B1
2,A2,B2
3,A3,B3


In [11]:
# 索引可以重复
df5 = pd.concat((df3,df4))
df5

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
1,A1,B1
2,A2,B2
3,A3,B3


In [12]:
# 所有的1,2都拿出来了
df5.loc[[1,2]]

,A,B
1,A1,B1
1,A1,B1
2,A2,B2
2,A2,B2


也可以选择忽略ignore_index，重新索引

In [13]:
# 忽略索引,重新分配索引
pd.concat((df3,df4),ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A1,B1
4,A2,B2
5,A3,B3


或者使用多层索引 keys  

concat([x,y],keys=['x','y'])

In [14]:
x = make_df(['a','b'],list('XY'))
y = make_df(['A','B'],list('XY'))
# kesys 在这里可以使我们合并zhi
pd.concat((x,y),keys=['x','y'])

X   Y
x a  Xa  Ya
  b  Xb  Yb
y A  XA  YA
  B  XB  YB

In [15]:
# {'inner', 'outer'}, default 'outer'
#   How to handle indexes on other axis(es)
pd.concat((df1,df2),sort=True,join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
1,B1,C1
2,B2,C2
3,B3,C3


============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

============================================

In [16]:
data = np.random.randint(0,150,size=(2,3))
ddd = DataFrame(data=data,index=['张三','李四'],columns=['语文','数学','英语'])
ddd

,语文,数学,英语
张三,140,140,72
李四,104,0,88


In [17]:
data = np.random.randint(0,150,size=(2,1))
ddd2 = DataFrame(data=data,index=['张三','李四'],columns=['计算机'])
ddd2

,计算机
张三,97
李四,69


In [18]:
# 新增计算机学科
ddd4 = pd.concat([ddd,ddd2],axis=1)
ddd4

,语文,数学,英语,计算机
张三,140,140,72,97
李四,104,0,88,69


In [19]:
# 新增王老五的成绩
data = np.random.randint(0,150,size=(1,4))
ddd3 = DataFrame(data=data,index=['王老五'],columns=['语文','数学','英语','计算机'])
ddd3

,语文,数学,英语,计算机
王老五,132,149,84,83


In [20]:
pd.concat((ddd4,ddd3))

,语文,数学,英语,计算机
张三,140,140,72,97
李四,104,0,88,69
王老五,132,149,84,83


### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有3种连接方式：

- 外连接：补NaN（默认模式）

In [21]:
df1 = make_df([1,3],list('AB'))
df2 = make_df([2,4],list('BC'))

In [22]:
# 默认是外链接out   会将所有的属性进行合并
pd.concat([df1,df2],sort=True)

,A,B,C
1,A1,B1,NaN
3,A3,B3,NaN
2,NaN,B2,C2
4,NaN,B4,C4


- 内连接：只连接匹配的项

In [23]:
# 内连接   合并了共有的属性,交集
pd.concat([df1,df2],join='inner')

,B
1,B1
3,B3
2,B2
4,B4


- 连接指定轴 join_axes

In [24]:
df3 = make_df([0,1,2],list('ACD'))
df4 = make_df([3,4,5],list('CDF'))

In [25]:
pd.concat([df3,df4],sort=True)

,A,C,D,F
0,A0,C0,D0,NaN
1,A1,C1,D1,NaN
2,A2,C2,D2,NaN
3,NaN,C3,D3,F3
4,NaN,C4,D4,F4
5,NaN,C5,D5,F5


In [26]:
# 以df3的轴为数据,例如下面,以某一个datafreme的列索引为新的列索引值,不指名,默认讲两个进行合并
pd.concat((df3,df4),join_axes=[df3.columns])

,A,C,D
0,A0,C0,D0
1,A1,C1,D1
2,A2,C2,D2
3,NaN,C3,D3
4,NaN,C4,D4
5,NaN,C5,D5


============================================

练习14：

    假设【期末】考试ddd2的成绩没有张三的，只有李四、王老五、赵小六的，使用多种方法级联

============================================

### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [27]:
df1 = make_df([0,1,2,3,4],['大众','福克斯'])
df2 = make_df([5,6,7,8,9],['大众','福克斯'])
# 用于数据的添加,合并
# concat方法属于 pandas模块
# append方法属于DataFrame里面的方法
df1.append(df2)

,大众,福克斯
0,大众0,福克斯0
1,大众1,福克斯1
2,大众2,福克斯2
3,大众3,福克斯3
4,大众4,福克斯4
5,大众5,福克斯5
6,大众6,福克斯6
7,大众7,福克斯7
8,大众8,福克斯8
9,大众9,福克斯9


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

使用pd.read...读取文件

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

###  1) 一对一合并

In [28]:
df1 = DataFrame({'emplyee':['po','Sara','Danis'],
                 'group':['sail','counting','marcketing']})
df1

,emplyee,group
0,po,sail
1,Sara,counting
2,Danis,marcketing


In [29]:
df2 = DataFrame({'emplyee':['po','Sara','Bush'],
                 'work_time':[2,3,1]})
df2

,emplyee,work_time
0,po,2
1,Sara,3
2,Bush,1


In [30]:
# 一对一合并
# 两个参数 ,左dataf,右dataf
pd.merge(df1,df2)

,emplyee,group,work_time
0,po,sail,2
1,Sara,counting,3


In [31]:
# 与concat区别
pd.concat((df1,df2),axis=1)

,emplyee,group,emplyee,work_time
0,po,sail,po,2
1,Sara,counting,Sara,3
2,Danis,marcketing,Bush,1


### 2) 多对一合并

In [32]:
df1 = DataFrame({'emplyee':['po','Sara','Danis'],
                 'group':['sail','counting','marcketing']})
# 有两个po
df2 = DataFrame({'emplyee':['po','po','Bush'],
                 'work_time':[2,3,1]})
display(df1,df2)

,emplyee,group
0,po,sail
1,Sara,counting
2,Danis,marcketing


,emplyee,work_time
0,po,2
1,po,3
2,Bush,1


In [33]:
# 多对一的情况
pd.merge(df1,df2)

,emplyee,group,work_time
0,po,sail,2
1,po,sail,3


### 3) 多对多合并

In [34]:
# 有两个po
df1 = DataFrame({'emplyee':['po','po','Danis'],
                 'group':['sail','counting','marcketing']})
# 有两个po
df2 = DataFrame({'emplyee':['po','po','Bush'],
                 'work_time':[2,3,1]})
display(df1,df2)

,emplyee,group
0,po,sail
1,po,counting
2,Danis,marcketing


,emplyee,work_time
0,po,2
1,po,3
2,Bush,1


In [35]:
# 在进行多对多合并时,每一个数据都没有放过
pd.merge(df1,df2)

,emplyee,group,work_time
0,po,sail,2
1,po,sail,3
2,po,counting,2
3,po,counting,3


### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用

In [36]:
df3 = DataFrame({'employee':['Po','Summer','Flower'],
                 'group':['sail','marketing','serch'],
                 'salary':[12000,10000,8000]})
df4 = DataFrame({'employee':['Po','Winter','Flower'],
                 'group':['marketing','marketing','serch'],
                 'work_time':[2,1,5]})
display(df3,df4)

,employee,group,salary
0,Po,sail,12000
1,Summer,marketing,10000
2,Flower,serch,8000


,employee,group,work_time
0,Po,marketing,2
1,Winter,marketing,1
2,Flower,serch,5


In [37]:
# 所有相同的才会合并
pd.merge(df3,df4)

,employee,group,salary,work_time
0,Flower,serch,8000,5


In [38]:
# 指定以某一列进行合并
pd.merge(df3,df4,on='employee')

,employee,group_x,salary,group_y,work_time
0,Po,sail,12000,marketing,2
1,Flower,serch,8000,serch,5


In [39]:
# 指定以某一列进行合并,suffixes是更名
pd.merge(df3,df4,on='group',suffixes=['_期中','_期末'])

,employee_期中,group,salary,employee_期末,work_time
0,Summer,marketing,10000,Po,2
1,Summer,marketing,10000,Winter,1
2,Flower,serch,8000,Flower,5


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不相等时使用

In [40]:
# 如果写错字了,属性不同部门写的不一样(employer,Team)
df3 = DataFrame({'employer':['Po','Summer','Flower'],
                 'Team':['sail','marketing','serch'],
                 'salary':[12000,10000,8000]})
df4 = DataFrame({'employee':['Po','Winter','Flower'],
                 'group':['marketing','marketing','serch'],
                 'work_time':[2,1,5]})
display(df3,df4)

,employer,Team,salary
0,Po,sail,12000
1,Summer,marketing,10000
2,Flower,serch,8000


,employee,group,work_time
0,Po,marketing,2
1,Winter,marketing,1
2,Flower,serch,5


In [41]:
# 正常合并
# pd.merge(df3,df4,left_on='employee',right_on='employee')

In [42]:
# 当一个表和另外一个表不一样的时候,可以使用这个
pd.merge(df3,df4,left_on='employer',right_on='employee')

,employer,Team,salary,employee,group,work_time
0,Po,sail,12000,Po,marketing,2
1,Flower,serch,8000,Flower,serch,5


In [43]:
# 先写的df3参数1,为左..df4参数2  为右
pd.merge(df3,df4,left_on='Team',right_on='group')

,employer,Team,salary,employee,group,work_time
0,Summer,marketing,10000,Po,marketing,2
1,Summer,marketing,10000,Winter,marketing,1
2,Flower,serch,8000,Flower,serch,5


============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

In [100]:
data = np.random.randint(50,150,size=(4,4))
columns = ['姓名','英语','数学','语文']
s_socrce = DataFrame(data=data,columns=columns)
s_socrce

,姓名,英语,数学,语文
0,85,57,64,134
1,132,104,91,113
2,103,61,72,123
3,120,85,88,107


In [110]:
s_socrce.姓名.loc[0:2] = ['张三','李四','王老五']
# s_socrce.drop([3],inplace=True)
s_socrce

,姓名,英语,数学,语文
0,张三,57,64,134
1,李四,104,91,113
2,王老五,61,72,123


In [112]:
data = np.random.randint(50,150,size=(2,4))
columns = ['姓名','英语','数学','语文']
s_socrce2 = DataFrame(data=data,columns=columns)
s_socrce2

,姓名,英语,数学,语文
0,89,149,79,72
1,102,83,63,70


In [116]:
s_socrce2.姓名 = ['张三','赵小六']
s_socrce2

,姓名,英语,数学,语文
0,张三,149,79,72
1,赵小六,83,63,70


### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

In [44]:
df1 = DataFrame({'age':[18,22,33],'height':[175,169,180]})
df2 = DataFrame({'age':[18,23,31],'weight':[65,70,80]})

In [45]:
# 内合并,默认的
pd.merge(df1,df2)

,age,height,weight
0,18,175,65


- 外合并 how='outer'：补NaN

In [46]:
# 外合并就是并集,所有的数据都会合并
pd.merge(df1,df2,how='outer')

,age,height,weight
0,18,175.0,65.0
1,22,169.0,NaN
2,33,180.0,NaN
3,23,NaN,70.0
4,31,NaN,80.0


- 左合并、右合并：how='left'，how='right'，

In [47]:
# 左合并,保留左边df1的数据进行合并
pd.merge(df1,df2,how='left')

,age,height,weight
0,18,175,65.0
1,22,169,NaN
2,33,180,NaN


In [48]:
# 右合并,保留右边df2的数据进行合并
pd.merge(df1,df2,how='right')

,age,height,weight
0,18,175.0,65
1,23,NaN,70
2,31,NaN,80


============================================

练习17：



1. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

In [49]:
display(df3,df4)

,employer,Team,salary
0,Po,sail,12000
1,Summer,marketing,10000
2,Flower,serch,8000


,employee,group,work_time
0,Po,marketing,2
1,Winter,marketing,1
2,Flower,serch,5


In [50]:
df3.columns = ['employee','group','salary']
display(df3,df4)

,employee,group,salary
0,Po,sail,12000
1,Summer,marketing,10000
2,Flower,serch,8000


,employee,group,work_time
0,Po,marketing,2
1,Winter,marketing,1
2,Flower,serch,5


In [51]:
pd.merge(df3,df4,on='employee',suffixes=['_李','_王'])

,employee,group_李,salary,group_王,work_time
0,Po,sail,12000,marketing,2
1,Flower,serch,8000,serch,5


============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

## 作业
## 3. 案例分析：美国各州人口数据分析

作业知识补充

In [52]:
# unique() 去重函数
s = Series(['Tom','Lucy','Tom','dancer','Lucy'])

In [53]:
s.unique()

array(['Tom', 'Lucy', 'dancer'], dtype=object)

In [54]:
n = DataFrame({'name':['Tom','Lucy','Tom','dancer','Lucy'],'age':[12,13,12,11,15]})
n

,name,age
0,Tom,12
1,Lucy,13
2,Tom,12
3,dancer,11
4,Lucy,15


In [55]:
# query 条件查询函数
n.query("name == 'Lucy' & age>14")  

,name,age
4,Lucy,15


首先导入文件，并查看数据样本

In [56]:
s_abb = pd.read_csv('./data/state-abbrevs.csv')
s_abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [57]:
s_pop = pd.read_csv('./data/state-population.csv')
s_pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [58]:
s_ares = pd.read_csv('./data/state-areas.csv')
s_ares.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


合并pop与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。

为了保留所有信息，使用外合并。

In [59]:
# 名字不一样时进行合并
s_abb_pop = pd.merge(s_abb,s_pop,left_on='abbreviation',right_on='state/region',how='outer')
s_abb_pop.head()

,state,abbreviation,state/region,ages,year,population
0,Alabama,AL,AL,under18,2012,1117489.0
1,Alabama,AL,AL,total,2012,4817528.0
2,Alabama,AL,AL,under18,2010,1130966.0
3,Alabama,AL,AL,total,2010,4785570.0
4,Alabama,AL,AL,under18,2011,1125763.0


去除abbreviation的那一列（axis=1）

In [60]:
s_abb_pop.drop('abbreviation',axis=1,inplace=True)

In [61]:
s_abb_pop

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0
5,Alabama,AL,total,2011,4801627.0
6,Alabama,AL,total,2009,4757938.0
7,Alabama,AL,under18,2009,1134192.0
8,Alabama,AL,under18,2013,1111481.0
9,Alabama,AL,total,2013,4833722.0


查看存在缺失数据的列。

使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [62]:
s_abb_pop.isnull().any()

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

查看缺失数据

根据数据是否缺失情况显示数据，如果缺失为True，那么显示

In [63]:
s_abb_pop.state

0       Alabama
1       Alabama
2       Alabama
3       Alabama
4       Alabama
5       Alabama
6       Alabama
7       Alabama
8       Alabama
9       Alabama
10      Alabama
11      Alabama
12      Alabama
13      Alabama
14      Alabama
15      Alabama
16      Alabama
17      Alabama
18      Alabama
19      Alabama
20      Alabama
21      Alabama
22      Alabama
23      Alabama
24      Alabama
25      Alabama
26      Alabama
27      Alabama
28      Alabama
29      Alabama
         ...   
2514        NaN
2515        NaN
2516        NaN
2517        NaN
2518        NaN
2519        NaN
2520        NaN
2521        NaN
2522        NaN
2523        NaN
2524        NaN
2525        NaN
2526        NaN
2527        NaN
2528        NaN
2529        NaN
2530        NaN
2531        NaN
2532        NaN
2533        NaN
2534        NaN
2535        NaN
2536        NaN
2537        NaN
2538        NaN
2539        NaN
2540        NaN
2541        NaN
2542        NaN
2543        NaN
Name: state, Length: 254

In [64]:
s_abb_pop.population

0         1117489.0
1         4817528.0
2         1130966.0
3         4785570.0
4         1125763.0
5         4801627.0
6         4757938.0
7         1134192.0
8         1111481.0
9         4833722.0
10        4672840.0
11        1132296.0
12        4718206.0
13        1134927.0
14        4569805.0
15        1117229.0
16        4628981.0
17        1126798.0
18        4530729.0
19        1113662.0
20        4503491.0
21        1113083.0
22        4467634.0
23        1120409.0
24        4480089.0
25        1116590.0
26        1121287.0
27        4430141.0
28        4452173.0
29        1122273.0
           ...     
2514     71946051.0
2515    282162411.0
2516     72376189.0
2517    279040181.0
2518    284968955.0
2519     72671175.0
2520    287625193.0
2521     72936457.0
2522    290107933.0
2523     73100758.0
2524    292805298.0
2525     73297735.0
2526    295516599.0
2527     73523669.0
2528    298379912.0
2529     73757714.0
2530    301231207.0
2531     74019405.0
2532    304093966.0


找到有哪些state/region使得state的值为NaN，使用unique()查看非重复值

In [84]:
# 得到不重复的  state/region
n1 = np.unique(s_abb_pop['state/region'])
n1

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
       'TX', 'USA', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype=object)

In [83]:
n2 = np.unique(s_abb['abbreviation'])
n2

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [91]:
# 得到带有nan的行索引
# s_abb_pop.isnull().any(axis=1)

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2514    False
2515    False
2516    False
2517    False
2518    False
2519    False
2520    False
2521    False
2522    False
2523    False
2524    False
2525    False
2526    False
2527    False
2528    False
2529    False
2530    False
2531    False
2532    False
2533    False
2534    False
2535    False
2536    False
2537    False
2538    False
2539    False
2540    False
2541    False
2542    False
2543    False
Length: 2544, dtype: bool

In [87]:
# table2.isnull().any(axis = 1) 得到有缺失值的行索引
# s_abb_pop.loc[s_abb_pop.isnull().any(axis=1)]

,state,state/region,ages,year,population
2448,Puerto Rico,PR,under18,1990,NaN
2449,Puerto Rico,PR,total,1990,NaN
2450,Puerto Rico,PR,total,1991,NaN
2451,Puerto Rico,PR,under18,1991,NaN
2452,Puerto Rico,PR,total,1993,NaN
2453,Puerto Rico,PR,under18,1993,NaN
2454,Puerto Rico,PR,under18,1992,NaN
2455,Puerto Rico,PR,total,1992,NaN
2456,Puerto Rico,PR,under18,1994,NaN
2457,Puerto Rico,PR,total,1994,NaN


In [67]:
def search(n1,n2):
    sta = []
    for i in n1:
        if i not in n2:
            sta.append(i)
    return sta

In [68]:
# 找出哪些值匹配不上abbreviation
arr = search(n1,n2)
arr
# PUERTO RICO PR 波多黎各  
# 美国

['PR', 'USA']

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！

记住这样清除缺失数据NaN的方法！

In [86]:
# 找到数据
cond1 = s_abb_pop['state/region'] == arr[0]
cond2 = s_abb_pop['state/region'] == arr[1]
s_abb_pop['state'][cond1] = 'Puerto Rico'
s_abb_pop['state'][cond2] = '美国'

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2514    False
2515    False
2516    False
2517    False
2518    False
2519    False
2520    False
2521    False
2522    False
2523    False
2524    False
2525    False
2526    False
2527    False
2528    False
2529    False
2530    False
2531    False
2532    False
2533    False
2534    False
2535    False
2536    False
2537    False
2538    False
2539    False
2540    False
2541    False
2542    False
2543    False
Name: state/region, Length: 2544, dtype: bool

In [70]:
s_abb_pop.isnull().any()

state           False
state/region    False
ages            False
year            False
population       True
dtype: bool

合并各州面积数据areas，使用外合并。

思考一下为什么使用外合并？



In [117]:
s_result = pd.merge(s_abb_pop,s_ares,how='outer')
s_result

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0
5,Alabama,AL,total,2011,4801627.0,52423.0
6,Alabama,AL,total,2009,4757938.0,52423.0
7,Alabama,AL,under18,2009,1134192.0,52423.0
8,Alabama,AL,under18,2013,1111481.0,52423.0
9,Alabama,AL,total,2013,4833722.0,52423.0


In [118]:
s_result.isnull().any()

state            False
state/region     False
ages             False
year             False
population        True
area (sq. mi)     True
dtype: bool

继续寻找存在缺失数据的列

In [72]:
n1 = np.unique(s_result['state'])
n2 = np.unique(s_ares['state'])
lis2 = search(n1,n2)
lis2

['美国']

我们会发现area(sq.mi)这一列有缺失数据，为了找出是哪一行，我们需要找出是哪个state没有数据

去除含有缺失数据的行

In [73]:
# 去除掉含有nan的那些行
s_result.dropna(axis=0,inplace=True)

查看数据是否缺失

In [74]:
s_result.isnull().any()

state            False
state/region     False
ages             False
year             False
population       False
area (sq. mi)    False
dtype: bool

找出2010年的全民人口数据,df.query(查询语句)

In [119]:
s_2010 = s_result.query("year == '2010' & ages == 'total'")

对查询结果进行处理，以state列作为新的行索引:set_index

In [122]:
# 设置datafrema中其中一列作为索引
# s_2010.set_index('state',inplace=True)
s_2010

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0
Colorado,CO,total,2010,5048196.0,104100.0
Connecticut,CT,total,2010,3579210.0,5544.0
Delaware,DE,total,2010,899711.0,1954.0
District of Columbia,DC,total,2010,605125.0,68.0


计算人口密度。注意是Series/Series，其结果还是一个Series。

In [77]:
s_result

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0
5,Alabama,AL,total,2011,4801627.0,52423.0
6,Alabama,AL,total,2009,4757938.0,52423.0
7,Alabama,AL,under18,2009,1134192.0,52423.0
8,Alabama,AL,under18,2013,1111481.0,52423.0
9,Alabama,AL,total,2013,4833722.0,52423.0


In [124]:
# 人口密度计算
pers = s_result["population"].div(s_result["area (sq. mi)"])
pers.name = 'density'
pers

0       21.316769
1       91.897221
2       21.573851
3       91.287603
4       21.474601
5       91.593900
6       90.760506
7       21.635389
8       21.202163
9       92.206131
10      89.137211
11      21.599222
12      90.002594
13      21.649410
14      87.171757
15      21.311810
16      88.300574
17      21.494344
18      86.426359
19      21.243767
20      85.906778
21      21.232722
22      85.222784
23      21.372470
24      85.460370
25      21.299620
26      21.389218
27      84.507583
28      84.927856
29      21.408027
          ...    
2514          NaN
2515          NaN
2516          NaN
2517          NaN
2518          NaN
2519          NaN
2520          NaN
2521          NaN
2522          NaN
2523          NaN
2524          NaN
2525          NaN
2526          NaN
2527          NaN
2528          NaN
2529          NaN
2530          NaN
2531          NaN
2532          NaN
2533          NaN
2534          NaN
2535          NaN
2536          NaN
2537          NaN
2538      

排序，并找出人口密度最高的五个州sort_values()

In [129]:
# 先把人口密度列插入进去
s_end = pd.concat((s_result,pers),axis=1,names='density')
# 升序排
s_end.sort_values(by='density').tail()


,state,state/region,ages,year,population,area (sq. mi),density
2539,美国,USA,total,2010,309326295.0,NaN,NaN
2540,美国,USA,under18,2011,73902222.0,NaN,NaN
2541,美国,USA,total,2011,311582564.0,NaN,NaN
2542,美国,USA,under18,2012,73708179.0,NaN,NaN
2543,美国,USA,total,2012,313873685.0,NaN,NaN


找出人口密度最低的五个州

In [126]:
# 先把人口密度列插入进去
s_end = pd.concat((s_result,pers),axis=1,names='density')
# 升序排
s_end.sort_values(by='density').head()

,state,state/region,ages,year,population,area (sq. mi),density
49,Alaska,AK,under18,1990,177502.0,656425.0,0.270407
51,Alaska,AK,under18,1991,182180.0,656425.0,0.277534
85,Alaska,AK,under18,2008,183124.0,656425.0,0.278972
83,Alaska,AK,under18,2007,184344.0,656425.0,0.280830
52,Alaska,AK,under18,1992,184878.0,656425.0,0.281644


要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息

## 回顾：Series/DataFrame运算与ndarray运算的区别

- Series与DataFrame没有广播，如果对应index没有值，则记为NaN；或者使用add的fill_value来补缺失值
- ndarray有广播，通过重复已有值来计算